<a href="https://colab.research.google.com/github/34maksuda/Fake_News_Detection_25/blob/main/Fake_News_Detection_Using_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install transformers

In [3]:
! pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.6/169.6 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of category-encoders to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 56.5 MB/s eta 0:00:00
   ━━━

In [4]:
import numpy as np
import pandas as pd
import pycaret
import transformers

In [5]:
from transformers import AutoModel, BertTokenizerFast
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch
import torch.nn as nn

In [6]:
device = torch.device("cuda")

In [7]:
from google.colab import drive

In [9]:
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
true_data = pd.read_csv('/content/drive/MyDrive/Research on NLP/Fake News/a1_True.csv')
fake_data = pd.read_csv('/content/drive/MyDrive/Research on NLP/Fake News/a2_Fake.csv')

In [ ]:
true_data.head()

In [ ]:
fake_data.head()

In [ ]:
len(true_data['text'][0])

In [ ]:
len(fake_data['text'][0])

In [ ]:
true_data['Target'] = ['True']*len(true_data)
fake_data['Target'] = ['Fake']*len(fake_data)

In [ ]:
true_data.head()

In [ ]:
fake_data.head()

In [ ]:
data = pd.concat([true_data, fake_data]).sample(frac=1)

In [ ]:
data.head()

In [ ]:
data = data.reset_index()

In [ ]:
data.head()

In [ ]:
data = data.drop(columns=['index'])

In [ ]:
data.head()

In [ ]:
data['label']=pd.get_dummies(data.Target)['Fake'].astype('int')

In [ ]:
data.head()

In [ ]:
label_size = [data['label'].sum(), len(data['label'])-data['label'].sum()]
label_size

In [ ]:
plt.pie(label_size, explode=[0.1, 0.1],colors=['firebrick', 'navy'], startangle=90, shadow=True, labels = ['Fake','True'], autopct='%1.1f%%')

In [ ]:
# Train-Validation-Test set split into 70:15:15 ratio
# Train-Temp split
train_text, temp_text, train_labels, temp_labels = train_test_split(data['text'], data['label'],
                                                                    random_state=2018,
                                                                    test_size=0.3,
                                                                    stratify=data['Target'])
# Validation-Test split
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels,
                                                                random_state=2018,
                                                                test_size=0.5,
                                                                stratify=temp_labels)

In [ ]:
# Load BERT model and tokenizer via HuggingFace Transformers
bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Plot histogram of the number of words in train data 'title'
seq_len = [len(title.split()) for title in train_text]
pd.Series(seq_len).hist(bins = 40,color='firebrick')
plt.xlabel('Number of Words')
plt.ylabel('Number of texts')

In [ ]:
# Majority of titles above have word length under 15. So, we set max title length as 15
MAX_LENGHT = 500
# Tokenize and encode sequences in the train set
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = MAX_LENGHT,
    pad_to_max_length=True,
    truncation=True
)
# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = MAX_LENGHT,
    pad_to_max_length=True,
    truncation=True
)
# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = MAX_LENGHT,
    pad_to_max_length=True,
    truncation=True
)

In [ ]:
# Convert lists to tensors
train_seq = torch.tensor(tokens_train['input_ids']).to(device)
train_mask = torch.tensor(tokens_train['attention_mask']).to(device)
train_y = torch.tensor(train_labels.tolist()).to(device)

val_seq = torch.tensor(tokens_val['input_ids']).to(device)
val_mask = torch.tensor(tokens_val['attention_mask']).to(device)
val_y = torch.tensor(val_labels.tolist()).to(device)

test_seq = torch.tensor(tokens_test['input_ids']).to(device)
test_mask = torch.tensor(tokens_test['attention_mask']).to(device)
test_y = torch.tensor(test_labels.tolist()).to(device)

In [ ]:
#Data Loader structure definition
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
batch_size = 8                                               #define a batch size
train_data = TensorDataset(train_seq, train_mask, train_y)    # wrap tensors
train_sampler = RandomSampler(train_data)                     # sampler for sampling the data during training
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
                                                              # dataLoader for train set
val_data = TensorDataset(val_seq, val_mask, val_y)            # wrap tensors
val_sampler = SequentialSampler(val_data)                     # sampler for sampling the data during training
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)
                                                              # dataLoader for validation set

In [ ]:
class BERT_Arch(nn.Module):
    def __init__(self, bert):
      super(BERT_Arch, self).__init__()
      self.bert = bert
      self.dropout = nn.Dropout(0.1)            # dropout layer
      self.relu =  nn.ReLU()                    # relu activation function
      self.fc1 = nn.Linear(768,512)             # dense layer 1
      self.fc2 = nn.Linear(512,2)               # dense layer 2 (Output layer)
      self.softmax = nn.LogSoftmax(dim=1)       # softmax activation function
    def forward(self, sent_id, mask):           # define the forward pass
      cls_hs = self.bert(sent_id, attention_mask=mask)['pooler_output']
                                                # pass the inputs to the model
      x = self.fc1(cls_hs)
      x = self.relu(x)
      x = self.dropout(x)
      x = self.fc2(x)                           # output layer
      x = self.softmax(x)                       # apply softmax activation
      return x
model = BERT_Arch(bert)
model = model.to(device)
# Defining the hyperparameters (optimizer, weights of the classes and the epochs)
# Define the optimizer
from transformers import AdamW
optimizer = AdamW(model.parameters(),
                  lr = 1e-5)          # learning rate
# Define the loss function
cross_entropy  = nn.NLLLoss()
# Number of training epochs
epochs = 2

In [ ]:
from tqdm import tqdm

def train():
    model.train()
    total_loss = 0

    # Wrap train_dataloader with tqdm for a progress bar
    train_loader = tqdm(train_dataloader, desc="Training", leave=False)
    for step, batch in enumerate(train_loader):  # iterate over batches
        batch = [r for r in batch]               # push the batch to GPU
        sent_id, mask, labels = [t.to(device) for t in batch]
        model.zero_grad()                        # clear previously calculated gradients
        preds = model(sent_id, mask)             # get model predictions for current batch
        loss = cross_entropy(preds, labels)      # compute loss between actual & predicted values
        total_loss += loss.item()                # add to the total loss
        loss.backward()                          # backward pass to calculate the gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # clip gradients
        optimizer.step()                         # update parameters

        # Update tqdm progress bar with the current batch loss
        train_loader.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_dataloader)  # compute average training loss
    return avg_loss

def evaluate():
    print("\nEvaluating...")
    model.eval()
    total_loss = 0

    # Wrap val_dataloader with tqdm for a progress bar
    val_loader = tqdm(val_dataloader, desc="Validation", leave=False)
    for step, batch in enumerate(val_loader):  # iterate over batches
        batch = [t for t in batch]             # push the batch to GPU
        sent_id, mask, labels = [t.to(device) for t in batch]
        with torch.no_grad():                  # deactivate autograd
            preds = model(sent_id, mask)       # model predictions
            loss = cross_entropy(preds, labels)  # compute validation loss
            total_loss += loss.item()

        # Update tqdm progress bar with the current batch loss
        val_loader.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(val_dataloader)  # compute average validation loss
    return avg_loss




In [ ]:
def train():
  model.train()
  total_loss, total_accuracy = 0, 0

  for step,batch in enumerate(train_dataloader):                # iterate over batches
    if step % 50 == 0 and not step == 0:                        # progress update after every 50 batches.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))
    batch = [r for r in batch]                                  # push the batch to gpu
    sent_id, mask, labels = [t.to(device) for t in batch]
    model.zero_grad()                                           # clear previously calculated gradients
    preds = model(sent_id, mask)                                # get model predictions for current batch
    loss = cross_entropy(preds, labels)                         # compute loss between actual & predicted values
    total_loss = total_loss + loss.item()                       # add on to the total loss
    loss.backward()                                             # backward pass to calculate the gradients
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)     # clip gradients to 1.0. It helps in preventing exploding gradient problem
    optimizer.step()                                            # update parameters
    preds=preds.detach().cpu().numpy()                          # model predictions are stored on GPU. So, push it to CPU

  avg_loss = total_loss / len(train_dataloader)                 # compute training loss of the epoch
                                                                # reshape predictions in form of (# samples, # classes)
  return avg_loss                                 # returns the loss and predictions

def evaluate():
  print("\nEvaluating...")
  model.eval()                                    # Deactivate dropout layers
  total_loss, total_accuracy = 0, 0
  for step,batch in enumerate(val_dataloader):    # Iterate over batches
    if step % 50 == 0 and not step == 0:          # Progress update every 50 batches.
                                                  # Calculate elapsed time in minutes.
                                                  # Elapsed = format_time(time.time() - t0)
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))
                                                  # Report progress
    batch = [t for t in batch]                    # Push the batch to GPU
    sent_id, mask, labels = [t.to(device) for t in batch]
    with torch.no_grad():                         # Deactivate autograd
      preds = model(sent_id, mask)                # Model predictions
      loss = cross_entropy(preds,labels)          # Compute the validation loss between actual and predicted values
      total_loss = total_loss + loss.item()
      preds = preds.detach().cpu().numpy()
  avg_loss = total_loss / len(val_dataloader)         # compute the validation loss of the epoch
  return avg_loss

In [ ]:
# Train and predict
best_valid_loss = float('inf')
train_losses = []  # empty lists to store training and validation loss of each epoch
valid_losses = []

for epoch in range(epochs):
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    train_loss = train()        # train model
    valid_loss = evaluate()     # evaluate model

    if valid_loss < best_valid_loss:  # save the best model
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'c2_new_model_weights.pt')

    train_losses.append(train_loss)  # append training and validation loss
    valid_losses.append(valid_loss)

    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')

In [ ]:
# Load weights of the best model
path = 'c2_new_model_weights.pt'
model.load_state_dict(torch.load(path))

# Transfer test data (test_seq and test_mask) to GPU
test_seq = test_seq.to(device)
test_mask = test_mask.to(device)

with torch.no_grad():
    # Get model predictions
    preds = model(test_seq, test_mask)
    preds = preds.detach().cpu().numpy()  # Move predictions to CPU for further processing

# Convert logits to class labels
preds = np.argmax(preds, axis=1)

# Move test_y to CPU and convert to NumPy array
test_y_np = test_y.cpu().numpy() # Added this line to move test_y to CPU

# Print classification report
print(classification_report(test_y_np, preds)) # Changed test_y to test_y_np

In [ ]:
# Testing on unseen data
unseen_news_text = [
    "Donald Trump Sends Out Embarrassing New Year’s Eve Message; This is Disturbing",     # Fake
    "WATCH: George W. Bush Calls Out Trump For Supporting White Supremacy",               # Fake
    "U.S. lawmakers question businessman at 2016 Trump Tower meeting: sources",           # True
    "Trump administration issues new rules on U.S. visa waivers"                          # True
]

# Tokenize and encode sequences in the test set
MAX_LENGTH = 15
tokens_unseen = tokenizer.batch_encode_plus(
    unseen_news_text,
    max_length=MAX_LENGTH,
    pad_to_max_length=True,
    truncation=True,
    return_tensors='pt'  # Return PyTorch tensors directly
)

# Move sequences and attention masks to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
unseen_seq = tokens_unseen['input_ids'].to(device)
unseen_mask = tokens_unseen['attention_mask'].to(device)

# Ensure the model is on the same device
model.to(device)

# Make predictions
with torch.no_grad():
    preds = model(unseen_seq, unseen_mask)
    preds = preds.detach().cpu().numpy()  # Move predictions back to CPU for processing

# Convert logits to class labels
preds = np.argmax(preds, axis=1)

# Output predictions
preds
